In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import os
import json

In [10]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [11]:
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_bbidding1.loc[:,"market_price_lag48h"] = df_bbidding1["market_price"].shift(96)
df_bbidding1.loc[:,"imbalance_price_lag48h"] = df_bbidding1["imbalance_price"].shift(96)
df_bbidding1.loc[:,"day_ahead_price_lag48h"] = df_bbidding1["day_ahead_price"].shift(96)
df_bbidding1.loc[:,"volume_lag48h"] = df_bbidding1["volume"].shift(96)
df_bbidding1.dropna(inplace=True)
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag48h,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h
96,2021-02-22 23:00:00+00:00,39.86,40.74,850.50,47,0.965926,0.611886,-0.000088,-9.562141e-08,-0.088636,...,1.146678e-09,2.732664e-20,40.952962,-0.658593,-4.207329,39.86000,26.46,32.75000,32.38,671.75
97,2021-02-22 23:30:00+00:00,25.88,40.74,755.65,48,0.965926,0.611886,-0.000088,-9.562141e-08,-0.088686,...,1.146678e-09,2.732664e-20,53.022213,14.302050,2.706285,25.88000,29.81,62.50000,32.38,905.45
98,2021-02-23 00:00:00+00:00,31.95,35.25,821.25,1,1.000000,0.598181,-2.668594,1.327215e+00,-7.243929,...,1.336674e+00,-5.844249e+01,7.936466,44.084019,84.017522,31.95000,7.67,11.79015,17.34,781.05
99,2021-02-23 00:30:00+00:00,32.09,35.25,852.25,2,1.000000,0.598181,-83.386905,-4.611063e+01,-92.411183,...,1.028806e+00,-6.628035e+01,50.825184,23.217360,89.462875,32.18992,9.82,12.29000,17.34,694.45
100,2021-02-23 01:00:00+00:00,30.75,32.74,789.00,3,0.965926,0.598181,-92.621997,-3.236547e+01,-51.507430,...,5.531250e+01,4.985395e+01,94.284233,109.047937,123.758802,30.55000,10.40,16.88000,14.25,847.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,28.044030,8.558464e+01,147.450968,...,2.575452e+02,3.120008e+02,389.209655,461.471346,565.748439,66.01000,0.00,70.00000,87.62,0.00
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-2.130944,4.406274e+01,80.500728,...,1.529461e+02,2.133404e+02,285.856261,352.111989,494.824570,66.01000,0.00,72.00000,85.84,0.00
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-53.850807,-3.103038e+01,1.674412,...,8.055552e+01,1.386240e+02,177.132223,250.514362,333.223882,66.01000,0.00,71.00000,85.84,0.00
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-42.706652,-3.898848e+01,19.495393,...,7.219972e+01,1.286610e+02,157.102188,214.652927,345.416021,66.01000,83.73,98.00000,82.02,194.45


In [12]:
# Original dataset
X = df_bbidding1[["market_price_lag48h","imbalance_price_lag48h","day_ahead_price_lag48h","volume_lag48h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
y = df_bbidding1["imbalance_price"].values  # Convert to numpy

# Step 1: Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 2: Further split the training set into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Step 3: Standardize the data (use only training data to fit the scaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [13]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        # Layer sizes from the best trial
        layer_sizes = [256, 448, 192, 96]
        dropout_rates = [0.12338360578207397, 0.2192742565593194, 0.15708417985889997, 0.253419888887539]

        # Define the layers
        self.fc1 = nn.Linear(input_dim, layer_sizes[0])
        self.fc2 = nn.Linear(layer_sizes[0], layer_sizes[1])
        self.fc3 = nn.Linear(layer_sizes[1], layer_sizes[2])
        self.fc4 = nn.Linear(layer_sizes[2], layer_sizes[3])
        self.fc5 = nn.Linear(layer_sizes[3], 1)  # Output layer

        # Dropouts
        self.dropout1 = nn.Dropout(dropout_rates[0])
        self.dropout2 = nn.Dropout(dropout_rates[1])
        self.dropout3 = nn.Dropout(dropout_rates[2])
        self.dropout4 = nn.Dropout(dropout_rates[3])

        # Activation function (Swish)
        self.swish = nn.SiLU()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # First layer with ReLU
        x = self.dropout1(x)         # First dropout
        x = self.swish(self.fc2(x))  # Second layer with Swish
        x = self.dropout2(x)         # Second dropout
        x = self.swish(self.fc3(x))  # Third layer with Swish
        x = self.dropout3(x)         # Third dropout
        x = self.swish(self.fc4(x))  # Fourth layer with Swish
        x = self.dropout4(x)         # Fourth dropout
        x = self.fc5(x)              # Output layer (no activation for raw outputs)
        return x



In [14]:
# y_train = y_train.values
# y_test = y_test.values

In [15]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Step 5: Initialize the model, loss function, and optimizer
input_dim = X_train_tensor.shape[1]
model = MLP(input_dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [16]:
import torch.nn as nn

# Define MAE loss
mae_loss = nn.L1Loss()

# Training parameters
num_epochs = 500
patience = 15  # Number of epochs to wait for improvement
min_relative_improvement = 0.000001  # 0.1% improvement threshold
best_val_loss = float('inf')
early_stopping_counter = 0

# Lists to store metrics
train_losses = []
val_losses = []
test_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = mae_loss(outputs, y_batch)  # Use MAE loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = mae_loss(val_outputs, y_val_tensor).item()  # Use MAE loss
        val_losses.append(val_loss)
        
        test_outputs = model(X_test_tensor)
        test_loss = mae_loss(test_outputs, y_test_tensor).item()  # Use MAE loss
        test_losses.append(test_loss)
    
    # Early stopping and relative improvement check
    if val_loss < best_val_loss:
        relative_improvement = (best_val_loss - val_loss) / best_val_loss
        if relative_improvement < min_relative_improvement and epoch > 10:
            print(f"Stopping due to small relative improvement: {relative_improvement:.6f}")
            break
        
        best_val_loss = val_loss
        early_stopping_counter = 0
        # Save best model
        torch.save(model.state_dict(), 'best_model.pth')

    else:
        early_stopping_counter += 1
    
    # Early stopping check
    if early_stopping_counter >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs")
        break
    
    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, "
              f"Test Loss: {test_loss:.4f}")

# Load best model after training
model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation
model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_loss = mae_loss(final_test_outputs, y_test_tensor).item()  # Use MAE loss
print(f"Final Test Loss: {final_test_loss:.4f}")


Epoch [10/500], Train Loss: 66.0726, Val Loss: 80.4423, Test Loss: 68.7473
Early stopping triggered after 20 epochs
Final Test Loss: 73.0408


C:\Users\paulh\AppData\Local\Temp\ipykernel_2864\4178530465.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [28]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])


In [22]:
data[1]

{'id': 'd57f64cd-0854-4d96-8e76-61284e004825',
 'participant_id': 'd742556b-2b48-493e-8959-7e6af7572f42',
 'prediction_date': '2024-10-08',
 'solution': {'market_day': '2024-10-08',
  'submission': [{'market_bid': 10.857783317565918,
    'timestamp': '2024-10-07T22:00:00+00:00',
    'probabilistic_forecast': {'10': 19,
     '20': 17,
     '30': 15,
     '40': 11,
     '50': 10,
     '60': 8,
     '70': 6,
     '80': 3,
     '90': 1}},
   {'market_bid': 10.857783317565918,
    'timestamp': '2024-10-07T22:30:00+00:00',
    'probabilistic_forecast': {'10': 19,
     '20': 17,
     '30': 15,
     '40': 11,
     '50': 10,
     '60': 8,
     '70': 6,
     '80': 3,
     '90': 1}},
   {'market_bid': 10.610013008117676,
    'timestamp': '2024-10-07T23:00:00+00:00',
    'probabilistic_forecast': {'10': 19,
     '20': 17,
     '30': 15,
     '40': 12,
     '50': 10,
     '60': 8,
     '70': 5,
     '80': 2,
     '90': 1}},
   {'market_bid': 10.610013008117676,
    'timestamp': '2024-10-07T23:30:00

In [29]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new

,prediction_date,timestamp,1,2,3,4,5,6,7,8,9
0,2024-10-04,2024-10-03T22:00:00+00:00,1355,487,963,1544,1330,867,326,1402,1357
1,2024-10-04,2024-10-03T22:30:00+00:00,788,584,1148,1320,1007,565,923,797,487
2,2024-10-04,2024-10-03T23:00:00+00:00,571,748,1001,1460,740,1017,1533,597,991
3,2024-10-04,2024-10-03T23:30:00+00:00,1476,1194,1512,334,1054,1572,744,1497,463
4,2024-10-04,2024-10-04T00:00:00+00:00,1352,1390,912,662,1505,1093,814,487,1340
...,...,...,...,...,...,...,...,...,...,...,...
763,2024-10-18,2024-10-18T19:30:00+00:00,142,208,301,360,409,451,485,537,617
764,2024-10-18,2024-10-18T20:00:00+00:00,185,242,327,349,407,450,509,551,623
765,2024-10-18,2024-10-18T20:30:00+00:00,207,270,340,383,429,454,520,563,633
766,2024-10-18,2024-10-18T21:00:00+00:00,225,299,357,404,461,494,546,581,620


In [34]:
df_api_new["timestamp"] = pd.to_datetime(df_api_new["timestamp"])
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.day/7)
df_api_new_merged

,prediction_date,timestamp,1,2,3,4,5,6,7,8,...,price_y,volume,timestamp_utc,settlement_date,settlement_period,imbalance_price,day_ahead_price,market_price,cos_hour,cos_day
0,2024-10-04,2024-10-03 22:00:00+00:00,1355,487,963,1544,1330,867,326,1402,...,71.32,1322.45,2024-10-03 22:00:00+00:00,2024-10-03,47.0,61.0,78.47,71.32,0.866025,-0.900969
1,2024-10-04,2024-10-03 22:30:00+00:00,788,584,1148,1320,1007,565,923,797,...,71.42,1035.65,2024-10-03 22:30:00+00:00,2024-10-03,48.0,50.0,78.47,71.42,0.866025,-0.900969
2,2024-10-04,2024-10-03 23:00:00+00:00,571,748,1001,1460,740,1017,1533,597,...,66.94,1378.55,2024-10-03 23:00:00+00:00,2024-10-04,1.0,93.5,71.36,66.94,0.965926,-0.900969
3,2024-10-04,2024-10-03 23:30:00+00:00,1476,1194,1512,334,1054,1572,744,1497,...,60.03,1539.90,2024-10-03 23:30:00+00:00,2024-10-04,2.0,93.5,71.36,60.03,0.965926,-0.900969
4,2024-10-04,2024-10-04 00:00:00+00:00,1352,1390,912,662,1505,1093,814,487,...,65.21,1818.40,2024-10-04 00:00:00+00:00,2024-10-04,3.0,93.0,72.80,65.21,1.000000,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,2024-10-18,2024-10-18 19:30:00+00:00,142,208,301,360,409,451,485,537,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.258819,-0.900969
771,2024-10-18,2024-10-18 20:00:00+00:00,185,242,327,349,407,450,509,551,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,-0.900969
772,2024-10-18,2024-10-18 20:30:00+00:00,207,270,340,383,429,454,520,563,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,-0.900969
773,2024-10-18,2024-10-18 21:00:00+00:00,225,299,357,404,461,494,546,581,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.707107,-0.900969


In [35]:
df_api_new_merged1 = df_api_new_merged[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag48h"] = df_api_new_merged1["market_price"].shift(96)
df_api_new_merged1.loc[:,"imbalance_price_lag48h"] = df_api_new_merged1["imbalance_price"].shift(96)
df_api_new_merged1.loc[:,"day_ahead_price_lag48h"] = df_api_new_merged1["day_ahead_price"].shift(96)
df_api_new_merged1.loc[:,"volume_lag48h"] = df_api_new_merged1["volume"].shift(96)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag48h,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h
96,2024-10-08 21:30:00+00:00,55.09,70.40,2360.40,46.0,0.707107,0.62349,18,17,16,...,10,8,4,3,1,55.09,71.32,61.00,78.47,1322.45
97,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.00000,19,17,15,...,10,8,6,3,1,100.00,71.42,50.00,78.47,1035.65
98,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.00000,19,17,15,...,10,8,6,3,1,66.00,66.94,93.50,71.36,1378.55
99,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.00000,19,17,15,...,10,8,5,2,1,66.00,60.03,93.50,71.36,1539.90
100,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.00000,19,17,15,...,10,8,5,2,1,66.00,65.21,93.00,72.80,1818.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,2024-10-14 19:30:00+00:00,203.10,91.30,318.15,42.0,0.258819,1.00000,32,34,36,...,40,42,44,46,49,124.71,203.10,124.71,91.30,318.15
483,2024-10-14 20:00:00+00:00,152.35,85.25,289.25,43.0,0.500000,1.00000,29,31,33,...,36,38,40,42,44,74.72,152.35,74.72,85.25,289.25
484,2024-10-14 20:30:00+00:00,118.88,85.25,541.20,44.0,0.500000,1.00000,32,34,36,...,40,42,44,46,49,74.72,118.88,74.72,85.25,541.20
485,2024-10-14 21:00:00+00:00,103.72,91.58,1282.20,45.0,0.707107,1.00000,35,37,40,...,44,46,49,51,53,102.00,103.72,102.00,91.58,1282.20


In [41]:
df_api_new_merged1 = df_api_new_merged1.groupby("timestamp_utc").last().reset_index()

In [42]:
X_test = df_api_new_merged1[["market_price_lag48h","imbalance_price_lag48h","day_ahead_price_lag48h","volume_lag48h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X_test = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged1["imbalance_price_predictions"] = final_test_outputs
df_api_new_merged1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,6,7,8,9,imbalance_price,market_price_lag48h,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h,imbalance_price_predictions
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.00000,19,17,15,...,8,6,3,1,100.00,71.42,50.00,78.47,1035.65,138.273224
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.00000,19,17,15,...,8,6,3,1,66.00,66.94,93.50,71.36,1378.55,140.597458
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.00000,19,17,15,...,8,5,2,1,66.00,60.03,93.50,71.36,1539.90,141.261703
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.00000,19,17,15,...,8,5,2,1,66.00,65.21,93.00,72.80,1818.40,149.052246
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.62349,18,16,14,...,7,5,3,1,65.00,64.79,93.00,72.80,1784.30,148.339706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,2024-10-14 19:30:00+00:00,203.10,91.30,318.15,42.0,0.258819,1.00000,32,34,36,...,42,44,46,49,124.71,203.10,124.71,91.30,318.15,192.768921
282,2024-10-14 20:00:00+00:00,152.35,85.25,289.25,43.0,0.500000,1.00000,29,31,33,...,38,40,42,44,74.72,152.35,74.72,85.25,289.25,170.120407
283,2024-10-14 20:30:00+00:00,118.88,85.25,541.20,44.0,0.500000,1.00000,32,34,36,...,42,44,46,49,74.72,118.88,74.72,85.25,541.20,157.457260
284,2024-10-14 21:00:00+00:00,103.72,91.58,1282.20,45.0,0.707107,1.00000,35,37,40,...,46,49,51,53,102.00,103.72,102.00,91.58,1282.20,153.563278


In [43]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged1["timestamp_utc"], y=df_api_new_merged1["imbalance_price"], mode='lines', name='Imbalance Price'))
fig.add_trace(go.Scatter(x=df_api_new_merged1["timestamp_utc"], y=df_api_new_merged1["imbalance_price_predictions"], mode='lines', name='Imbalance Price Predictions'))
fig.update_layout(title='Imbalance Price and Predictions', xaxis_title='Timestamp', yaxis_title='Imbalance Price')
fig.show()

In [45]:
#MAE calculation for imbalance price predictions
mae = np.mean(np.abs(df_api_new_merged1["imbalance_price"] - df_api_new_merged1["imbalance_price_predictions"]))
print(f"MAE for imbalance price predictions: {mae:.4f}")

MAE for imbalance price predictions: 103.2994
